In [3]:
import pandas as pd
import gensim
from gensim import corpora
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from gensim import corpora
from gensim.models import LdaMulticore, Word2Vec
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import pandas as pd
#from pyLDAvis import gensim_models
#pyLDAvis.enable_notebook()
from pyTCTK import TextNet
import warnings
warnings.filterwarnings("ignore")
from wordcloud import WordCloud

In [4]:
df = pd.read_csv('data/df_topics.csv')
df = df.drop('Unnamed: 0', axis=1)

In [5]:
df

,W_movie ID,Movie name,Release date,Countries,Languages,Genres,summary,Country_extract,Region
0,32986669,Robbery Under Arms,1907-11-02,"{""/m/0chghy"": ""Australia""}","{""/m/06ppq"": ""Silent film""}","{""/m/06ppq"": ""Silent film"", ""/m/07s9rl0"": ""Dra...",Key scenes of the film included the branding o...,['Australia'],['Asia and Pacific']
1,7870349,Dr. Jekyll and Mr. Hyde,1908-03-07,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",Dr. Jekyll and Mr. Hyde began with the raising...,['United States of America'],['America']
2,29391146,The Black Viper,1908-07-25,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film""}","{""/m/06ppq"": ""Silent film""}",A thug accosts a girl as she leaves her workpl...,['United States of America'],['America']
3,28777800,The Englishman and the Girl,1910-02-17,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",A small town's drama group is preparing for a ...,['United States of America'],['America']
4,13254122,What the Daisy Said,1910-07-11,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",Two farm sisters are feeling romantic and loo...,['United States of America'],['America']
...,...,...,...,...,...,...,...,...,...
8136,14299402,The Blade Spares None,NaN,"{""/m/03h64"": ""Hong Kong""}","{""/m/0653m"": ""Standard Mandarin""}","{""/m/02kdv5l"": ""Action"", ""/m/01hmnh"": ""Fantasy...","Ho Li-Chun, an attractive but powerful swordsw...",['Hong Kong'],['Asia and Pacific']
8137,16705375,The Seed,NaN,"{""/m/09c7w0"": ""United States of America""}","{""/m/02h40lc"": ""English Language""}","{""/m/01jfsb"": ""Thriller"", ""/m/02hmvc"": ""Short ...",The Seed follows the events surrounding a home...,['United States of America'],['America']
8138,11971266,La Guerre des tuques,NaN,"{""/m/0d060g"": ""Canada""}","{""/m/064_8sq"": ""French Language""}","{""/m/0hj3myq"": ""Children's/Family"", ""/m/0hj3mt...",The film involves a huge snowball fight betwee...,['Canada'],['America']
8139,11515305,Buio Omega,NaN,"{""/m/03rjj"": ""Italy""}","{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","{""/m/03npn"": ""Horror""}","Anna Völkl, the fiance of taxidermist Frank Wy...",['Italy'],['\u200bEurop Region']


In [6]:
nltk.download('wordnet')
def preprocess(text):
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()
    text = text.lower()
    stop_free = " ".join([word for word in text.split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

df['clean_summary'] = df['summary'].apply(preprocess)


[nltk_data] Downloading package wordnet to /Users/bina/nltk_data...


In [7]:
regions = df['Region'].unique()

for region in regions:
    region_df = df[df['Region'] == region]
    texts = region_df['clean_summary'].apply(lambda x: x.split())

    # Creating the term dictionary of our corpus, where every unique term is assigned an index.
    dictionary = corpora.Dictionary(texts)

    # Converting list of documents (corpus) into Document Term Matrix using the dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in texts]

    # Creating the LDA model
    lda_model = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50)

    # Print the topics for each region
    print(f"Topics for Region: {region}")
    for idx, topic in lda_model.print_topics(-1):
        print('Topic: {} \nWords: {}'.format(idx, topic))
    print("\n")

    


Topics for Region: ['Asia and Pacific']
Topic: 0 
Words: 0.006*"father" + 0.006*"get" + 0.005*"go" + 0.005*"him" + 0.005*"tell" + 0.005*"love" + 0.005*"one" + 0.005*"find" + 0.004*"come" + 0.004*"take"
Topic: 1 
Words: 0.004*"one" + 0.004*"back" + 0.004*"find" + 0.003*"attack" + 0.003*"take" + 0.003*"two" + 0.003*"fight" + 0.003*"him" + 0.003*"goku" + 0.003*"get"
Topic: 2 
Words: 0.004*"one" + 0.004*"kill" + 0.003*"find" + 0.003*"him" + 0.003*"men" + 0.003*"king" + 0.003*"however" + 0.003*"terrorist" + 0.003*"police" + 0.003*"take"
Topic: 3 
Words: 0.006*"get" + 0.006*"police" + 0.005*"him" + 0.005*"father" + 0.005*"find" + 0.005*"love" + 0.005*"one" + 0.004*"go" + 0.004*"take" + 0.004*"life"
Topic: 4 
Words: 0.004*"find" + 0.004*"godzilla" + 0.004*"one" + 0.003*"back" + 0.003*"attack" + 0.003*"take" + 0.003*"him" + 0.003*"two" + 0.003*"fight" + 0.003*"kill"


Topics for Region: ['America']
Topic: 0 
Words: 0.007*"dr" + 0.006*"sun" + 0.005*"one" + 0.005*"kenner" + 0.004*"film" + 0.004*

In [8]:
# ... [previous code for setting up DataFrame and preprocessing]

output_filename = "topic_modeling_results.txt"

with open(output_filename, "w") as file:
    regions = df['Region'].unique()

    for region in regions:
        region_df = df[df['Region'] == region]
        texts = region_df['clean_summary'].apply(lambda x: x.split())

        # Creating the term dictionary of our corpus, where every unique term is assigned an index.
        dictionary = corpora.Dictionary(texts)

        # Converting list of documents (corpus) into Document Term Matrix using the dictionary prepared above.
        doc_term_matrix = [dictionary.doc2bow(doc) for doc in texts]

        # Creating the LDA model
        lda_model = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=5, id2word=dictionary, passes=50)

        # Write the topics for each region to the file
        file.write(f"Topics for Region: {region}\n")
        for idx, topic in lda_model.print_topics(-1):
            file.write('Topic: {} \nWords: {}\n'.format(idx, topic))
        file.write("\n")
